<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/GradientDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터가 계속 쌓여도 이전 데이터와 훈련된 모델을 버리지 않고, 새로운 데이터에 대해 조금씩 더 훈련하는 것을 '점진적 학습'이라고 한다.
# 대표적으로 확률적 경사 하강법이 있다.

# 확률적 경사 하강법 = 훈련 세트에서 랜덤하게 하나의 샘플을 선택하여 경사를 조금 내려간다. 그 다음 훈련 세트에서 랜덤하게 또 다른 샘플을 하나 선택하여 경사를 조금 내려간다.
# 이런식으로 전체 샘플을 모두 사용할 때까지 계속한다.
# 만약, 모든 샘플을 다 사용해도 원하는 위치까지 하강하지 못하면, 다시 처음부터 시작한다. (훈련 세트에 모든 샘플을 다시 채워넣는 것이다.) 그 다음 다시 랜덤하게 하나의 샘플을 선택해 이어서 경사를 내려간다.
# 이렇게 만족할만한 위치에 도달할 때까지 계속 내려간다. 확률적 경사하강법에서 훈련 세트를 한 번 모두 사용하는 과정을 에포크 'epoch'라고 한다.

# 1개씩 말고 무작위로 몇개의 샘플을 선택해서 경사를 내려가면, 미니배치 경사하강법이라고 한다.
# 전체 샘플을 이용해 내려갈 경우는 배치 경사 하강법이라고 한다.

In [ ]:
# 이러한 확률적 경사하강법을 반드시 사용해야 하는 알고리즘으로 '신경망 알고리즘'이 있다.
# 신경망은 많은 데이터를 사용하므로 한 번에 모든 데이터를 사용하기 어렵기 때문이다.

In [ ]:
# 손실함수 'loss function'이란 어떤 문제에서 머신러닝 알고리즘이 얼마나 엉터리인가를 측정하는 기준이다.
# 즉, 이 값은 작을수록 좋다. 하지만 어떤 값이 최솟값인지는 알지 못하기 때문에, 경사하강법을 사용하여 조금씩 찾아나간다.
# 그리고 손실함수는 미분 가능해야 한다.

# 경사적 하강법을 이용해 손실함수의 최솟값을 찾아내는 게 핵심이다.
# 손실함수의 종류 중 하나로 '로지스틱 손실 함수'와 '크로스 엔트로피 손실 함수'가 있다.
# 이진 분류에는 '로지스틱 손실 함수'를, 다중분류에는 '크로스 엔트로피 손실 함수'를 사용한다.

In [ ]:
import pandas as pd
fish=pd.read_csv('https://bit.ly/fish_csv_data')

In [ ]:
fish_input=fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target=fish['Species'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input,test_input,train_target,test_target=train_test_split(fish_input,fish_target,random_state=42)

In [ ]:
# 훈련 세트와 테스트 세트의 특성을 표준화 및 전처리
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input) # 훈련세트에서 학습한 통계값으로 테스트 세트도 변환해야 함.

In [ ]:
from sklearn.linear_model import SGDClassifier


In [ ]:
# loss 매개 변수는 손실함수 종류를 지정, 여기서는 로지스틱 손실 함수를 지정한다.
sc=SGDClassifier(loss='log_loss',max_iter=10,random_state=42)
sc.fit(train_scaled,train_target)

print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))

In [ ]:
# 정확도가 낮은 것으로 보아 반복횟수가 부족한 것 같다. 더 늘리자.
# 모델을 이어서 훈련할 때는 partial_fit()을 사용한다. 호출할 때마다 1에포크씩 이어서 훈련할 수 있다.
sc.partial_fit(train_scaled,train_target)

print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))

In [ ]:
# 확률적 경사 하강법을 사용한 모델은 에포크 횟수에 따라 과소적합이나 과대 적합이 될 수 있다.
# 에포크가 부족하면, 너무 적게 훈련해서 과소적합이 될 확률이 높다.
# 반면 에포크가 지나치게 많으면, 너무 많이 훈련해서 과대적합될 확률이 높다.
# 과대적합이 시작되기 전에 훈련을 멈추는 것을 '조기 종료'라고 한다.

In [ ]:
import numpy as np

sc=SGDClassifier(loss='log_loss',random_state=42)

train_score=[]
test_score=[]
classes=np.unique(train_target)

In [ ]:
for _ in range(0,300):
    sc.partial_fit(train_scaled,train_target,classes=classes)
    train_score.append(sc.score(train_scaled,train_target))
    test_score.append(sc.score(test_scaled,test_target))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_score)
plt.plot(test_score)

plt.xlabel('epoch')
plt.ylabel('aacuracy')
plt.show()

# 파란색이 훈련 세트, 주황색이 테스트세트다. 100 에포크일 때가 적절한 반복 횟수로 보인다.

In [ ]:
# 아예 다시 100회 처음부터 훈련
sc=SGDClassifier(loss='log_loss',max_iter=100,tol=None,random_state=42)# tol 매개변수는 향상될 최솟값을 뜻함. 점수 도달하면 반복을 멈춤.
sc.fit(train_scaled,train_target)

print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))